In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-yfpc78jn
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-yfpc78jn
  Resolved https://github.com/ibm-granite-community/utils to commit ccbb2307ce43170395c921b3305c9d13b9d24a85
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for ibm_granite_community-utils: filename=ibm_granite_community_utils-0.1.dev45-py3-none-any.whl size=8315 sha256=1989d2d23499ef3ac

In [2]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    # model="ibm-granite/granite-3.0-8b-instruct",
    model="ibm-granite/granite-8b-code-instruct-128k",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_length":100, "temperature":0.2},
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


In [3]:
def zeroshot_prompt(context, question):
    """
    Creates a zero-shot prompt for the model, where the model acts as a seasoned programmer.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform

    Returns:
    - str, the formatted prompt
    """
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to help build an online bookstore. Begin by creating UI components using the Jupiter Notebook Widgets for interactivity.
    The ouput code should not exceed 100 Tokens. Do not use GridBoxLayout attribute in the output.
    Write Python code to address each specific user request. Return only the code as output.

    Task: {context}

    Question: {question}
    """
    return prompt


In [ ]:
def perform_operation(x, y, op):
    if op == 'add':
        return x + y
    else:
        return "Invalid operation"

x = int(input("Enter the value for x: "))
y = int(input("Enter the value for y: "))
op = input("Enter the operator (+ or -): ")

result = perform_operation(x, y, op)
print(f"Result of {x} {op} {y}: {result}")

In [4]:
def get_answer_using_zeroshot(context, question):
    """
    Generates the response from the model based on a zero-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer

    Returns:
    - str, the generated result from the model
    """
    prompt = zeroshot_prompt(context, question)
    result = model.invoke(prompt)

    return result

In [6]:
context = "Design and develop an online bookstore UI components with minimalistic theme"
question = "Create the landing page for users visiting my bookstore. The landing page should display a Welcome message `Welcome to IBM Community Learners Reader's Club` along with a catalog of books titles as tiles. Add the image in the triple back ticks section as widget image ```https://books.toscrape.com/media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg``` as the "

# Generate and display the UI code for the landing page
result = get_answer_using_zeroshot(context, question)
print(f"Generated Code:\n{result}")

Generated Code:
Here is the code for creating the landing page for users visiting my bookstore:
```python
import ipywidgets as widgets
from IPython.display import display
# Create a landing page with a welcome message and a catalog of books
welcome_message = widgets.HTML(value="Welcome to IBM Community Learners Reader's Club")
display(welcome_message)
# Create a catalog of books as tiles
books = ["Book 1", "Book 2", "Book 3", "Book 4", "Book 5"]
book_tiles = widgets.GridBox(children=[widgets.Label(book) for book in books], layout=widgets.Layout(grid_template_columns='repeat(auto-fit, minmax(200px, 1fr))'))
display(book_tiles)
# Add an image to the landing page
image = widgets.Image(value="https://books.toscrape.com/media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg")
display(image)
```
This code creates a landing page with a welcome message, a catalog of books as tiles, and an image. The catalog of books is displayed as a grid of tiles, with each tile representing a book. The image

In [ ]:
examples = [
    {
        "question": "Create a UI for my landing page showing a catalog for my online bookstore",
        "context": "Simple Catalog Tile with Borders and Shadows",
        "output": """
import ipywidgets as widgets
from IPython.display import display

# Create a container for the book catalog
container = widgets.VBox()

# Create styled book tiles
book_tiles = [
    widgets.HTML(value="
        <div style='background-color: #f5f5f5; padding: 15px; margin: 10px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);'>
            <h2 style='color: #333;'>Book Title 1</h2>
            <p style='color: #333;'><b>Author:</b> Author Name 1</p>
            <p style='color: #333;'><b>Price:</b> $10.00</p>
        </div>"),
    widgets.HTML(value="
        <div style='background-color: #f5f5f5; padding: 15px; margin: 10px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);'>
            <h2 style='color: #333;'>Book Title 2</h2>
            <p style='color: #333;'><b>Author:</b> Author Name 2</p>
            <p style='color: #333;'><b>Price:</b> $15.00</p>
        </div>"),
]

container.children = book_tiles
display(container)
"""
    },
    {
        "question": "Add a search bar to my online bookstore's landing page with a placeholder and search icon",
        "context": "Search Bar with Placeholder Text and Search Button",
        "output": """
import ipywidgets as widgets
from IPython.display import display

# Create search bar with placeholder text and search icon
search_bar = widgets.HBox([
    widgets.Text(
        placeholder="Search for a book...",
        layout=widgets.Layout(width="250px", padding="5px"),
        style={"description_width": "0px"}
    ),
    widgets.Button(
        icon="search",
        tooltip="Search",
        layout=widgets.Layout(width="40px", padding="5px")
    )
])

display(search_bar)

"""
    },
    {
        "question": "Design an enhanced catalog UI with alternating tile colors for my online bookstore",
        "context": "Enhanced Book Catalog with Alternating Tile Colors and Font Styling",
        "output": """
import ipywidgets as widgets
from IPython.display import display

# Create search bar with placeholder text and search icon
search_bar = widgets.HBox([
    widgets.Text(
        placeholder="Search for a book...",
        layout=widgets.Layout(width="250px", padding="5px"),
        style={"description_width": "0px"}
    ),
    widgets.Button(
        icon="search",
        tooltip="Search",
        layout=widgets.Layout(width="40px", padding="5px")
    )
])

display(search_bar)

"""
    }

]

In [ ]:
def few_shot_prompt(context, question):
    example_prompts = "\n\n".join(
        f"Context: {ex['context']}\nOutput:\n{ex['output']}" for ex in examples
    )
    # prompt = f"""
    # You are an experienced programmer with 15 years of experience writing full-stack applications.
    # Your task is to help build an online bookstore with Tkinter components.
    # Examples:
    # {example_prompts}

    # Now, {context}
    # """
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to help build an online bookstore with Jupyter Notebook widgets for UI components.
    Begin by creating UI components using the Jupiter Notebook Widgets for interactivity.
    The ouput code should not exceed 100 Tokens. Do not use GridBoxLayout attribute in the output.
    Write Python code to address each specific user request. Return only the code as output.

    Examples:
    {example_prompts}

    Now, {context}
    Question: {question}
    """
    return prompt

In [ ]:
# Function to get answer using few-shot prompting
def get_answer_using_few_shot(context, question):
    prompt = few_shot_prompt(context, question)
    result = model.invoke(prompt)
    return result

# Define context and question for generation
# context = "create a UI code for the landing page for my online bookstore with colored header and items arranged in rows"
# question = "Create Tkinter code for an online bookstore with header and items in a row."
context = "Design and develop an online bookstore with user-friendly UI components."
question = "Create the landing page for users visiting my bookstore. The landing page should display a catalog of book titles as tiles and include a free-text search bar with a search icon."

# Run few-shot prompt
response = get_answer_using_few_shot(context, question)
print(response)

Here is the code to create the landing page for the bookstore:
```python
import ipywidgets as widgets
from IPython.display import display
# Create a container for the book catalog
container = widgets.VBox()
# Create styled book tiles
book_tiles = [
 widgets.HTML(value="
 <div style='background-color: #f5f5f5; padding: 15px; margin: 10px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);'>
 <h2 style='color: #333;'>Book Title 1</h2>
 <p style='color: #333;'><b>Author:</b> Author Name 1</p>
 <p style='color: #333;'><b>Price:</b> $10.00</p>
 </div>"),
 widgets.HTML(value="
 <div style='background-color: #f5f5f5; padding: 15px; margin: 10px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);'>
 <h2 style='color: #333;'>Book Title 2</h2>
 <p style='color: #333;'><b>Author:</b> Author Name 2</p>
 <p style='color: #333;'><b>Price:</b> $15.00</p>
 </div>"),
]
container.children = book_tiles
# Create search bar with placeholder text and search icon
search_bar = widg

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
def generate_cot_prompt(context, question):
    prompt = f"""
    You are an experienced UI developer tasked with designing a user-friendly, styled navigation bar for an online bookstore in Jupyter Notebook.
    The ouput code should not exceed 100 Tokens. Do not use GridBoxLayout attribute in the output.
    Write Python code to address each specific user request.
    Please follow these Chain of Thought steps:

    Step 1: Define the navigation items and layout based on context.
    Step 2: Create buttons for each navigation item (e.g., Home, Catalog, Cart, Account) using ipywidgets.
    Step 3: Arrange buttons in a horizontal layout for a navigation bar.
    Step 4: Add custom CSS styles for hover effects, padding, and layout alignment.
    Step 5: Combine all components and render the complete navigation bar.

    Context: {context}

    Question: {question}
    provide a step-by-step breakdown of the code that is needed to complete the task.
    """

#Now, proceed to provide a step-by-step breakdown along with the code needed to complete the task.
    return prompt

In [ ]:
# Function to get answer using few-shot prompting
def get_answer_using_cot(context, question):
    prompt = generate_cot_prompt(context, question)
    result = model.invoke(prompt)
    return result

# Define context and question for generation
# context = "create a UI code for the landing page for my online bookstore with colored header and items arranged in rows"
# question = "Create Tkinter code for an online bookstore with header and items in a row."
context = "Design and develop an online bookstore with user-friendly UI components."
question = "Create the landing page for users visiting my bookstore. The landing page should display a catalog of book titles as tiles and include a free-text search bar with a search icon."

# Run few-shot prompt
response = get_answer_using_cot(context, question)
print(response)

Sure, here's a step-by-step breakdown of the code needed to create the landing page for users visiting your bookstore:
1. Import the necessary libraries:
```python
import ipywidgets as widgets
from IPython.display import display
```
2. Define the book titles as a list of strings:
```python
book_titles = ['Book 1', 'Book 2', 'Book 3', 'Book 4', 'Book 5']
```
3. Create a function to display the book titles as tiles:
```python
def display_books(book_titles):
 for title in book_titles:
 button = widgets.Button(description=title)
 display(button)
```
4. Create a search bar with a search icon:
```python
search_bar = widgets.Text(placeholder='Search for a book...')
search_icon = widgets.Image(value='search_icon.png')
search_box = widgets.HBox(children=[search_bar, search_icon])
display(search_box)
```
5. Combine the book tiles and search bar into a single layout:
```python
layout = widgets.VBox(children=[widgets.Label('Catalog'), widgets.HBox(children=book_titles), search_box])
display(layout